In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import sqlite3

In [3]:
import requests

username = 'yorifuji'
params = {'username': username}

response = requests.get('https://zenn.dev/api/articles', params=params)
articles_json = response.json()["articles"]

class Article:
    def __init__(self, json_data):
        self.id = json_data["id"]
        self.title = json_data["title"]
        self.url = f'https://zenn.dev/yorifuji/articles/{json_data["slug"]}'
        self.created_at = json_data["created_at"]
        self.published = json_data["published"]

published_articles = []
for json_data in articles_json:
    if json_data["published"]:
        published_articles.append(Article(json_data))

sorted_articles = sorted(published_articles, key=lambda article: article.created_at)

for article in sorted_articles:
    print(f'{article.id},{article.title},{article.url},{article.created_at}')


KeyError: 'published'

In [4]:
conn = sqlite3.connect('data.db')
c = conn.cursor()

while True:
    response = requests.get('https://zenn.dev/api/articles')
    articles_json = response.json()["articles"]

    # pathを取得
    paths = [article["path"] for article in articles_json]
    print(paths)
    for path in paths:

        # DBの中に同じpathがあるかどうかを確認
        c.execute("SELECT * FROM url WHERE url=?", (path,))
        # ある場合は操作をパス
        if c.fetchone() is not None:
            continue

        # pathをDBに保存
        c.execute("INSERT INTO url(url) VALUES (?)", (path,))

        # tagsを取得
        url = f'https://zenn.dev/{path}'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # div class="View_topics__OVMdM"の中のaタグを取得
        tags = [tag.text for tag in soup.select('div.View_topics__OVMdM a')]

        # 最後の一個は消す
        tags.pop()
        print(tags)

        # tagsをDBに保存
        for tag in tags:
            # 文字列の最初に#がついていたら消す
            if tag[0] == "#":
                tag = tag[1:]
            c.execute("INSERT INTO node(node) VALUES (?)", (tag,))
        
        # nodeのidが1000を超えたらbreak
        c.execute("SELECT * FROM node")
        if len(c.fetchall()) > 1000:
            break
    # nodeのidが1000を超えたらbreak
    c.execute("SELECT * FROM node")
    if len(c.fetchall()) > 1000:
        break

['/kagan/articles/html-text-level-semantics', '/frontend_news/articles/904cda6410d44e', '/ficilcom/articles/app_router_registant_to_changes', '/pluck/articles/42d30febace0a7', '/entaku/articles/8cb384245fc302', '/vim_jp/articles/2023-08-21-vim-accomplished-plugins-result-2', '/shundeveloper/articles/8be490a0327cb1', '/kyome/articles/d21dbf28287927', '/fraim/articles/produce-measurement-with-attributes-from-tracing', '/airiswim/articles/1643db802618de', '/bs_kansai/articles/8618aaa04504b7', '/ikefukurou777/articles/6518f3facf64eb', '/top/articles/55e445cd49378d', '/otoyo/articles/8b9b93db4e2265', '/togetine/articles/6c9db2db7a9dd9', '/myuon/articles/bd2a0e134cdd24', '/ken_ty/articles/417f58aaa67ec3', '/ryoyoshii/articles/ce928c3f89af55', '/flutteruniv_dev/articles/bfc3b4be6b31ae', '/minedia/articles/61a9a107211422', '/ms5/articles/f9173936299b1d', '/ficilcom/articles/agile-development-experience', '/bs_kansai/articles/6508e7b37e137f', '/egstock_inc/articles/1dd8cc2d38f2ef', '/msakuta/ar

In [22]:
url = "https://zenn.dev/topics"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# div class="TopicCardList_name__vcZH2"を全て取得
div = soup.find_all("div", class_="TopicCardList_name__vcZH2")

# タグは消す
tags = [d.text.replace("#", "") for d in div]
print(len(tags))


60
57


### API
https://zenn.dev/api/articles

### docs
https://zenn-dev.github.io/zenn-docs-for-developers/basics/overview